In [2]:
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
import sys
warnings.filterwarnings('ignore')

In [3]:
def syspath(path) :
    if path not in sys.path :
        sys.path.append(path)
    else : pass

In [4]:
syspath('./dataset/mod')

from Preprocessing import Encoder

In [5]:
# 데이터 로드 후 Label Encoding 실시
odf = pd.read_pickle('./dataset/00_feature_with_label.pkl')
en = Encoder(odf)
df = en.label()

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
confusion_matrix, f1_score, roc_auc_score

def get_clf_eval(y_test, pred, pred_proba):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    # ROC AUC는 pred 대신 pred_proba
    roc_score=roc_auc_score(y_test,pred_proba) # pred_proba

    print('오차행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f},\
    f1 score:{3:.4f}, ROC AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    

def do_lgbm(X_train, X_test, y_train, y_test, param=1, param_compare=0):
    if param == 1:
        lgbm_clf = LGBMClassifier(n_estimators=100, random_state=156, n_jobs=-1)
    elif param == 2:
        lgbm_clf = LGBMClassifier(n_estimators=100, random_state=156, num_leaves=128,
                              n_jobs=-1)
    elif param == 3:
        lgbm_clf = LGBMClassifier(n_estimators=100, random_state=156, num_leaves=128,
                              max_depth=64, n_jobs=-1)
    elif param == 4:
        lgbm_clf = LGBMClassifier(n_estimators=100, random_state=156,
                              max_depth=32, min_child_samples=64,
                              num_leaves=96, n_jobs=-1)

    evals = [(X_test, y_test)]
    
    if param_compare==0:
        lgbm_clf.fit(X_train,y_train, early_stopping_rounds=10,eval_set=evals, \
                      eval_metric='logloss',verbose=True)
    elif param_compare==1:
        lgbm_clf.fit(X_test, y_test, early_stopping_rounds=10,eval_set=evals, \
                      eval_metric='logloss',verbose=True)
    w_preds = lgbm_clf.predict(X_test)
    w_preds_proba = lgbm_clf.predict_proba(X_test)[:,1]

    get_clf_eval(y_test, w_preds, w_preds_proba)

 #### 아래와 같이 smote 수행 시 정밀도가 큰 폭으로 떨어지는 것을 볼 수 있음. 이유는 하단에서 추가 설명

    오차행렬
    [[422328   4213]
    [   339   2422]]
    정확도:0.9894, 정밀도:0.3650, 재현율:0.8772,    f1 score:0.5155, ROC AUC:0.9948

In [ ]:
ndf = df.copy()
X_feature = ndf.drop('LABEL', axis=1)
y_label = ndf.LABEL

X_train, X_test, y_train, y_test = train_test_split(X_feature,
                                                    y_label, test_size = 0.2, random_state=50)

sm = SMOTE(random_state=50, n_jobs=-1)

X_resampled1, y_resampled1 = sm.fit_resample(X_train,list(y_train))
do_lgbm(X_resampled1, X_test, y_resampled1, y_test, 1)

[1]	valid_0's binary_logloss: 0.627763	valid_0's binary_logloss: 0.627763
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.573975	valid_0's binary_logloss: 0.573975
[3]	valid_0's binary_logloss: 0.5287	valid_0's binary_logloss: 0.5287
[4]	valid_0's binary_logloss: 0.489978	valid_0's binary_logloss: 0.489978
[5]	valid_0's binary_logloss: 0.457329	valid_0's binary_logloss: 0.457329
[6]	valid_0's binary_logloss: 0.425109	valid_0's binary_logloss: 0.425109
[7]	valid_0's binary_logloss: 0.398664	valid_0's binary_logloss: 0.398664
[8]	valid_0's binary_logloss: 0.37549	valid_0's binary_logloss: 0.37549
[9]	valid_0's binary_logloss: 0.350315	valid_0's binary_logloss: 0.350315
[10]	valid_0's binary_logloss: 0.327875	valid_0's binary_logloss: 0.327875
[11]	valid_0's binary_logloss: 0.309711	valid_0's binary_logloss: 0.309711
[12]	valid_0's binary_logloss: 0.291749	valid_0's binary_logloss: 0.291749
[13]	valid_0's binary_logloss: 0.275784	valid_0's bin

In [ ]:
# svmsmote는 실행 시간이 오래걸려(최소 6시간) 별도 pkl 파일 중 일부를 추출해 놓음
import joblib
df_svm = joblib.load('/content/drive/MyDrive/NA1/Project2_Final/dataset/SVMSMOTE1.pkl')
df_svm

,TRD_NAME_2,TRD_COUNTRY_2,TRD_ADDR_2,CUS_TOTAL_NUMBER_OF_PACKAGES_6,CUS_REF_NO_7,CON_TIN_8,CON_NAME_8,CON_COUNTRY_8,CON_ADDR_8,IMP_TRADING_COUNTRY_11,VAL_FINANCIAL_VALUE_12,VAL_CURRENCY_12,IMP_CNT_OF_DISPATCH_EXP_CD_15,IMP_COUNTRY_OF_ORIGIN_16,DEL_DELIVERY_TERM_CODE_20,DEL_PLACE_OF_DELIVERY_20,TOT_CURRENCY_22,TOT_FINANCIAL_VALUE_22,IMP_EXCHANGE_RATE_23,IMP_INLAND_TRANSPORT_MODE_25,IMP_TRANSPORT_MODE_AT_BODR_26,OFF_CODE_30,GDS_GOODS_DESCRIPTION_31,GDS_ITEM_NUMBER_32,COM_COMBINED_NOMENCLATURE_33,IDG_COUNTRY_OF_ORIGIN_34,GDS_GROSS_MASS_35,PRF_PREFERENCE_CODE_1,PRF_PREFERENCE_CODE_2,PRF_PREFERENCE_CODE_3,PRF_PREFERENCE_CODE_4,IDG_PROCEDURE_REQUESTED_37,IDG_PREVIOUS_PROCEDURE_37,IDG_ADD_NATIONAL_PROC_37,IDG_NET_MASS_38,FIN_FINANCIAL_VALUE_42,COR_FINANCIAL_VALUE,COR_CURRENCY,STC_FINANCIAL_VALUE_46,CAL_TYPE_OF_TAX_47,CAL_TAX_BASE_47,CAL_RATE_OF_TAX_47,CAL_ADDITIONAL_RATE_OF_TAX_47,PAM_FINANCIAL_VALUE_47,CAL_METHOD_OF_PAYMENT_47,IMP_DATE_OF_DECLARATION_54,REP_TIN_54,PERSON_NAME_54,GEND_REFERENCE_54,GEND_ISSUE_DATE_54,ACCEPTANCE_DATE
0,14807,77,26000,3700.000000,183141,90014719,52671,18,53135,2,1.974912e+05,0,77,77,4,3955,22,22200.000000,8.896000,30.0,30.0,762317,417001,1,701909000,98,185000.000000,1,11,0,10,40,0,0,185000.000000,22200.000000,197491.200000,0,22200.000000,10.000000,197491.200000,70.000000,1,622.720000,1,20180427,165,664,898,20140606.0,201326
1,1776,97,31007,8.000000,10293,20018266,54273,18,26559,106,2.780548e+05,0,96,97,12,6207,22,26200.000000,8.808000,40.0,40.0,762115,523801,11,8803200000,0,175.000000,1,11,4,10,61,21,0,165.000000,2000.000000,22147.720000,0,2515.000000,30.000000,9387.130000,18.000000,0,1689.680000,0,20170823,20,377,373,20160405.0,160483
2,59403,62,6022,7623.000000,219013,30027518,51512,18,32135,1,4.683638e+05,0,62,25,2,1124,8,39453.000000,10.675200,30.0,30.0,762518,457311,3,3307100000,35,148.000000,1,11,4,10,40,0,0,148.000000,385.000000,4510.510000,0,478.000000,20.000000,4510.510000,6.500000,0,293.180000,1,20190315,69,109,393,20150907.0,95701
3,5193,101,30293,1.000000,6725,20027506,53341,18,37912,110,1.133987e+04,0,100,98,3,2797,22,1170.000000,9.692200,40.0,40.0,762115,429087,2,3822000000,123,0.840000,1,11,4,10,40,0,0,0.750000,420.000000,4070.720000,0,420.000000,30.000000,4335.320000,18.000000,0,780.360000,0,20191015,22,159,391,20150731.0,99863
4,55684,25,14684,127.000000,4405,20009202,44075,18,27552,29,2.575348e+05,0,25,25,2,2797,22,26945.000000,7.880500,40.0,40.0,762115,148686,1,2208909100,44,300.000000,0,4,2,5,40,0,2,275.000000,4375.000000,59400.540000,0,7538.000000,10.000000,257534.790000,140.000000,1,1103.270000,4,20170110,36,1111,884,20121227.0,63088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412011,58799,20,35236,9603.000000,100449,120032833,15548,18,59205,23,3.671868e+06,0,20,20,2,3112,22,389035.000000,9.438400,30.0,30.0,762230,297088,14,6402998943,28,4747.612744,1,11,4,10,40,0,0,4747.612744,9495.225487,89619.737489,0,9495.225487,16.879822,83651.797712,14.063947,0,11175.139823,0,20190228,400,651,52,20130507.0,199505
3412012,24460,62,6057,3881.472316,179535,405021569,68604,18,77004,104,1.627815e+05,0,62,24,1,5970,8,14386.469254,9.956121,20.0,20.0,762312,201194,0,8708736498,35,1321.499635,1,11,0,10,40,0,0,1321.499635,280.694645,16201.870927,0,1840.507413,18.721056,16177.258938,3.337373,0,741.674027,1,20174469,179,113,886,20170606.0,247325
3412013,10458,20,951,9582.746262,104992,640007645,56471,18,61866,23,6.332331e+05,0,20,20,3,5426,22,65322.066617,9.693939,30.0,30.0,762235,199728,35,9608200000,28,1340.886107,1,11,4,10,40,0,0,1322.848993,827.824746,8024.837306,0,827.824746,33.726804,9781.374033,19.118041,0,1871.212023,0,20191112,252,151,557,20080622.0,80991
3412014,58799,20,35236,7182.000000,100448,120032833,15548,18,59205,23,2.970963e+06,0,20,20,2,3112,22,314774.000000,9.438400,30.0,30.0,762230,485270,38,7216894442,28,16489.885749,1,1

## svmsmote 및 smote 등 오버샘플링을 사용하지 않는 이유

- 하단 2개의 결과 셀에서 볼수 있는 것처럼, 원본 데이터프레임 중 TRD_COUNTRY_2의 유니크 값의 범위가 0~ 108임
- 반면에 svmsmote가 적용된 새로운 데이터프레임 중 TRD_COUNTRY_2의 유니크 값의 범위는 -18 ~ 130임
=> 즉 명목형 변수임에도 새로운 값이 창조됨

이로인해 오히려 정밀도가 하락하는 현상이 발생했고, 이러한 이유로 본 프로젝트에서는 오버샘플링을 사용하지 않음.

In [ ]:
# 원본 데이터프레임 중 TRD_COUNTRY_2의 unique 리스트
origin_unique_list = df.TRD_COUNTRY_2.unique()
origin_unique_list.sort()
origin_unique_list

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108])

In [ ]:
# svmsmote를 수행한 새로운 데이터프레임 중 TRD_COUNTRY_2의 unique 리스트
svmsmote_unique_list = df_svm.TRD_COUNTRY_2.unique()
svmsmote_unique_list.sort()
svmsmote_unique_list

array([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,
        -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,
         8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
       125, 126, 127, 128, 129, 130])